# 01｜重新出发，让我们学会和AI说话

## 1 获取 API Key

步骤略，自行找攻略。将生成的 API KEY 设置到环境变量中。

## 2 搭建开发环境

### 本地环境

使用 Python + JupyterNoteBook，IDE 使用 PyCharm，然后安装下面的依赖即可：

```shell
conda create --name py310 python=3.10
conda activate py310
conda install -c conda-forge jupyterlab
conda install -c conda-forge ipywidgets
conda install -c conda-forge openai
```

当然也可以使用 pip 来安装这些依赖。

### 通过 Colab 使用 JupyterLab

[Colab](https://colab.research.google.com/)，不想折腾本地环境的，还有一个选择，就是使用 Google 提供的叫做 Colab 的线上 Python Notebook 环境。

另外，即使你有本地环境，也建议创建一个 Colab 账号，因为 Colab 可以让你免费使用一些 GPU 的资源，在你需要使用 GPU 尝试训练一些深度学习模型，而又没有一张比较好的显卡的时候，就可以直接使用它。另一方面，Colab 便于你在网络上把自己撰写的 Python 代码分享给其他人。

Colab 已经是一个 Python Notebook 的环境了。只需要安装 openai 的库即可：

```shell
!pip install openai
%env OPENAI_API_KEY=在这里写你获取到的ApiKey
```

## 3 体验并测试 OpenAI 的 API

尝试运行下面代码

In [1]:
# 准备一下环境
from jproperties import Properties
import openai
import os

configs = Properties()
with open("../private.properties", 'rb') as config_file:
    configs.load(config_file)

# 设置 OpenAI 的 Key
openai.api_key = configs.get("open_key").data

# 解决 SSLError，参考 https://www.zhihu.com/question/588395420/answer/2928476066。
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

In [2]:
import openai

COMPLETION_MODEL = "text-davinci-003"

test_prompt = """
Consideration product : 工厂现货PVC充气青蛙夜市地摊热卖充气玩具发光蛙儿童水上玩具

1. Compose human readable product title used on Amazon in english within 20 words.
2. Write 5 selling points for the products in Amazon.
3. Evaluate a price range for this product in U.S.

Output the result in json format with three properties called title, selling_points and price_range
"""

def get_response(prompt):
    completions = openai.Completion.create(
        engine=COMPLETION_MODEL,
        prompt=prompt,
        max_tokens=512,
        n=1,
        stop=None,
        temperature=0.0,
    )
    message = completions.choices[0].text
    return message


print(get_response(test_prompt))


{
    "title": "PVC Inflatable Glow-in-the-Dark Frog Water Toy for Kids Night Market Stall Hot Sale",
    "selling_points": [
        "Made of durable PVC material",
        "Inflatable design for easy storage and transport",
        "Glow-in-the-dark feature for night time fun",
        "Perfect for water play and pool parties",
        "Great for kids of all ages"
    ],
    "price_range": "$10 - $20"
}
